In [36]:
import os
from netmiko import ConnectHandler
from netmiko.exceptions import NetmikoAuthenticationException, NetmikoTimeoutException

# --- 1. Define connection parameters ---
# Replace with your actual VPCS connection information
# Assume VPCS can be accessed via 192.168.1.3 port 5000 in GNS3
device = {
    # Try 'generic_telnet'
    "device_type": "generic_telnet", 
    
    # Replace with VPCS IP or GNS3 access IP
    "host": "192.168.1.3", 
    
    # Replace with Telnet port. In GNS3, this is usually the device console port
    "port": 5002, 
    
    # VPCS usually doesn't have username and password, but some GNS3 environments may require it.
    # If your VPCS doesn't have credentials, you can comment out or set to empty strings
    "username": "", 
    "password": "", 
    
    # VPCS may need time to startup, set longer connection timeout
    "timeout": 30,

}

# Your regular expression PC1> ...PC2> .... PC10>
PROMPT_REGEX = r"PC\d+>"

# --- 2. Command line execution ---
#test_command = "ip 10.0.0.2/24 10.0.0.1"  # VPCS command example
test_command = "ping 10.0.0.1"
try:
    print(f"Trying to connect using {device['device_type']} to {device['host']}:{device['port']}...")
    
    # Establish connection
    net_connect = ConnectHandler(**device)
    current_prompt = net_connect.find_prompt().strip()

    print(f"Device prompt identified as: { current_prompt}")

    print("✅ Connection successful!")
    
    # Send command
    # Note: VPCS CLI characteristics may require using send_command_timing
    # But we'll start with standard send_command first
    output = net_connect.send_command_timing(
        test_command,
        read_timeout=15, # 增加命令的等待时间
        delay_factor=2, # 针对此命令增加延迟
        #expect_string=PROMPT_REGEX
        )
    
    print(f"\n--- {test_command} output ---")
    print(output)
    print("------------------------\n")
    
    # Disconnect connection
    net_connect.disconnect()

except NetmikoAuthenticationException:
    print(f"❌ Authentication failed! Please check username and password (if VPCS requires it).")
except NetmikoTimeoutException:
    print(f"❌ Connection timeout! Please check if IP, port {device['port']} are correct and device is started.")
except Exception as e:
    print(f"❌ Other error occurred: {e}")

Trying to connect using generic_telnet to 192.168.1.3:5002...
Device prompt identified as: PC1>
✅ Connection successful!

--- ping 10.0.0.1 output ---

84 bytes from 10.0.0.1 icmp_seq=1 ttl=255 time=0.885 ms
84 bytes from 10.0.0.1 icmp_seq=2 ttl=255 time=1.000 ms
84 bytes from 10.0.0.1 icmp_seq=3 ttl=255 time=0.898 ms
84 bytes from 10.0.0.1 icmp_seq=4 ttl=255 time=0.908 ms
84 bytes from 10.0.0.1 icmp_seq=5 ttl=255 time=0.980 ms


------------------------

